# Phys555 Assignment 2 Question 2
Karlee Zammit - V00823093

Q2 - Use  the Q1 regression data set for the following:
A) Use the linear regression model in the module 'Class04-Jan19.' Change the hyperparameters to get the best predictions. Explain the details and discuss the accuracy/performance of the model (i.e., associated plots and metrics). 
B) Do the same with KNN regression. What is the best K for your data? Which model (A or B) is more accurate (higher performance)?

# Introduction: Linear Regression and KNN

In [1]:
# Import the necessary packages
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
%matplotlib inline
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from pandas.plotting import scatter_matrix    
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import sys
from scipy.interpolate import interp1d
from matplotlib import pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
print('Done!')

Done!


# Functions

## Linear Regression Function

In [2]:
# Use the same linear regression model as in Q2 so the results are comparable, for the data keeping only n_components 
def linear_regression(X_tr, X_va, Y_tr, Y_va, xlim, ylim, textx, texty, text2x, text2y):
    reg = linear_model.SGDRegressor(loss='squared_error', penalty='L2', alpha=0.001,
                                   max_iter=2000, eta0=.001, tol=0.0001, learning_rate='constant', n_iter_no_change=5)

    # Fit the model on the PCA data
    reg.fit(X_tr,Y_tr)  # fit the model with training set

    #'predictions for training and validation sets'
    Y_tr_pred = reg.predict(X_tr)  
    Y_va_pred = reg.predict(X_va)

    plt.figure(1)
    plt.plot(Y_tr,Y_tr_pred,'ob')
    plt.plot(Y_va,Y_va_pred,'.r')

    plt.plot(np.arange(xlim,ylim,.1),  np.arange(xlim,ylim,.1),'-k')
    plt.xlabel('X')
    plt.ylabel('Predicted X')
    plt.legend(['Training', 'Validation'], loc='best')
    # plt.xlim([0,2])
    # plt.ylim([0,2])

    #Statistical information regarding training and validation predictions
    mu_lr = np.mean(Y_tr-Y_tr_pred)
    median_lr = np.median(Y_tr-Y_tr_pred)
    sigma_lr = np.std(Y_tr-Y_tr_pred)

    muv_lr = np.mean(Y_va-Y_va_pred)
    medianv_lr = np.median(Y_va-Y_va_pred)
    sigmav_lr = np.std(Y_va-Y_va_pred)

    textstr = '$\mu=%.4f$\n$\mathrm{med}=%.4f$\n$\sigma=%.4f$'%(mu_lr, median_lr, sigma_lr)
    textstrv = '$\mu=%.4f$\n$\mathrm{med}=%.4f$\n$\sigma=%.4f$'%(muv_lr, medianv_lr, sigmav_lr)

    plt.text(textx,texty,textstr, color='b',fontsize=15)
    plt.text(text2x,text2y,textstrv, color='r',fontsize=15)
    plt.title('Linear Regression')
    
    plt.figure(2)
    plt.hist(Y_tr_pred-Y_tr,20,color='b',histtype='step',density=True,label='training')
    plt.hist(Y_va_pred-Y_va,20,color='r',histtype='step',density=True,label='validation')
    plt.xlabel('Predicted - real')
    plt.ylabel('Probability (density)')
    plt.legend()
    
    plt.figure(3)
    plt.scatter(Y_tr,Y_tr_pred-Y_tr,label='training',color='b',alpha=.3)
    plt.scatter(Y_va,Y_va_pred-Y_va,label='test',color='r',alpha=.3)
    plt.xlabel('Y (real)')
    plt.ylabel('Y (predicted) - Y (real)')
    plt.plot([xlim,ylim],[0,0],'y')
    plt.legend()

## KNN Function

In [3]:
def nearest_neighbour(X_tr, X_va, Y_tr, Y_va, xlim, ylim, textx, texty, text2x, text2y):    
    KNN = KNeighborsRegressor(n_neighbors=5, weights='uniform')  # set the hyperparameters
    KNN.fit(X_tr,Y_tr)  # train it

    #'predictions for training and validation sets'
    Y_tr_pred= KNN.predict(X_tr)  
    Y_va_pred= KNN.predict(X_va)

    plt.figure(1)
    plt.plot(Y_tr,Y_tr_pred,'ob')
    plt.plot(Y_va,Y_va_pred,'.r')

    plt.plot(np.arange(xlim,ylim,.1),  np.arange(xlim,ylim,.1),'-k')
    plt.xlabel('X')
    plt.ylabel('Predicted X')
    plt.legend(['Training', 'Validation'])

    #Statistical information regarding training and validation predictions
    mu = np.mean(Y_tr-Y_tr_pred)
    median = np.median(Y_tr-Y_tr_pred)
    sigma = np.std(Y_tr-Y_tr_pred)

    muv = np.mean(Y_va-Y_va_pred)
    medianv = np.median(Y_va-Y_va_pred)
    sigmav = np.std(Y_va-Y_va_pred)

    textstr = '$\mu=%.4f$\n$\mathrm{med}=%.4f$\n$\sigma=%.4f$'%(mu, median, sigma)
    textstrv = '$\mu=%.4f$\n$\mathrm{med}=%.4f$\n$\sigma=%.4f$'%(muv, medianv, sigmav)

    plt.text(textx,texty,textstr, color='b',fontsize=15)
    plt.text(text2x,text2y,textstrv, color='r',fontsize=15)
    plt.title('KNN Regression')
    
    plt.figure(2)
    plt.hist(Y_tr_pred-Y_tr,20,color='b',histtype='step',density=True,label='training')
    plt.hist(Y_va_pred-Y_va,20,color='r',histtype='step',density=True,label='validation')
    plt.xlabel('Predicted - real')
    plt.ylabel('Probability (density)')
    plt.legend()
    
    plt.figure(3)
    plt.scatter(Y_tr,Y_tr_pred-Y_tr,label='training',color='b',alpha=.3)
    plt.scatter(Y_va,Y_va_pred-Y_va,label='test',color='r',alpha=.3)
    plt.xlabel('Y (real)')
    plt.ylabel('Y (predicted) - Y (real)')
    plt.plot([xlim,ylim],[0,0],'y')
    plt.legend()
    

## Split and Normalize Data Function

In [4]:
def split_data(X,Y):
    # Split the input and target data into test and train, with 75% of the data going to training, and 25% of the data going to testing
    X_tr, X_va, Y_tr, Y_va = train_test_split(X,Y.values.ravel(),test_size=0.25)

    # Print the shape of the split data
    print ('training set == ',np.shape(X_tr),np.shape(Y_tr),',, validation set == ', np.shape(X_va),np.shape(Y_va))
    
    # Normalize the data, as was done in Q2
    scaler_S= StandardScaler().fit(X_tr)  # line #2
    X_tr_Norm= scaler_S.transform(X_tr) # line # 3
    X_va_Norm= scaler_S.transform(X_va)  # Line #4
    
    n_column = 2

    # Plot a figure of the normalized training and validation set to ensure they represent the same distribution and spread
    
    fig = plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(X_tr_Norm[:,n_column])
    plt.title('Training set')
    plt.ylabel('N')
    plt.xlabel("X"+str(n_column))

    plt.subplot(1, 2, 2)
    plt.hist(X_va_Norm[:,n_column])
    plt.title('Validation set')
    plt.ylabel('N')
    plt.xlabel("X"+str(n_column))
    
    return X_tr_Norm, X_va_Norm, Y_tr, Y_va

# Sachs Harbour: Linear Regression and KNN

In [5]:
df_SH = pd.read_excel('Ambient Sound Data Sachs Harbour 2015-2016.xlsx')
df_SH = df_SH.drop(['Deployment', 'Year', 'Month', 'Day', 'Hour', 'DateTime', 'Ice'], axis=1)
df_SH=df_SH.dropna(axis=0)

# Set temperature as the target variable
var = ['Temperature']

# Drop temperature out of the weather data, and set it as X
X = df_SH.loc[:,df_SH.columns.drop(var)]
print(X.columns)

# Set the target (Y) to be temperature
Y = df_SH[var].copy()
print(Y.columns)

X_tr_Norm, X_va_Norm, Y_tr, Y_va = split_data(X,Y)

FileNotFoundError: [Errno 2] No such file or directory: 'Ambient Sound Data Sachs Harbour 2015-2016.xlsx'

In [ ]:
linear_regression(X_tr_Norm, X_va_Norm, Y_tr, Y_va, -40, 30, 45, 10, 45, -10)

In [ ]:
nearest_neighbour(X_tr_Norm, X_va_Norm, Y_tr, Y_va, -40, 30, 45, 10, 45, -10)

# Discussion

# References

Bishop, C. M. (2006). Pattern recognition and machine learning. In Pattern recognition and machine learning. Springer.

https://www.kaggle.com/datasets/budincsevity/szeged-weather

# Appendix: Extra Datasets

## Szeged Weather

In [ ]:
# Read in the csv data and perform the same data mining steps as was done in question 1
weather = pd.read_csv(r'weatherHistory.csv', delimiter=',')
weather=weather.dropna(axis=0)
weather = weather.drop(['Formatted Date', 'Summary', 'Daily Summary', 'Precip Type', 'Loud Cover', 'Visibility (km)'], axis=1)
weather_mined = weather.loc[ weather['Pressure (millibars)'] != 0]
weather_mined = weather_mined.loc[weather_mined['Humidity'] != 0]
weather_mined.hist(figsize=(10, 8), bins=30, edgecolor="black")
plt.subplots_adjust(hspace=0.7, wspace=0.4)

In [ ]:
# Set temperature as the target variable
var = ['Temperature (C)']

# Drop temperature out of the weather data, and set it as X
X = weather_mined.loc[:, weather_mined.columns.drop(var)]
print(X.columns)

# Set the target (Y) to be temperature
Y = weather_mined[var].copy()
print(Y.columns)

X_tr_Norm, X_va_Norm, Y_tr, Y_va = split_data(X,Y)

In [ ]:
linear_regression(X_tr_Norm, X_va_Norm, Y_tr, Y_va, -20, 40, 45, 10, 45, -10)

In [ ]:
nearest_neighbour(X_tr_Norm, X_va_Norm, Y_tr, Y_va, -20, 40, 45, 10, 45, -10)

## Beans

In [ ]:
beans = pd.read_excel(r'Dry_Bean_Dataset.xlsx')
beans = beans.drop(['Class'], axis=1)

# Set temperature as the target variable
var = ['Perimeter']

# Drop temperature out of the weather data, and set it as X
X = beans.loc[:, beans.columns.drop(var)]
print(X.columns)

# Set the target (Y) to be temperature
Y = beans[var].copy()
print(Y.columns)

X_tr_Norm, X_va_Norm, Y_tr, Y_va = split_data(X,Y)

In [ ]:
linear_regression(X_tr_Norm, X_va_Norm, Y_tr, Y_va, 450, 2000, 2100, 800, 2100, 1400)

In [ ]:
nearest_neighbour(X_tr_Norm, X_va_Norm, Y_tr, Y_va, 450, 2000, 2100, 800, 2100, 1400)